In [143]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import networkx as nx
import folium
from scipy.spatial import Voronoi
from math import floor, dist
# for requirements: install fastparquet 

## 1. Import data 

### Handover and EnB data

In [15]:
PATH_DATA = '..\\Processed_data\\HO_AGG_16-22_4G_coords_reduced.snappy.parquet'
agg_df_reduced = pd.read_parquet(PATH_DATA, engine='fastparquet')
ho_df = agg_df_reduced.loc[agg_df_reduced.srcLocInfo != agg_df_reduced.dstLocInfo]
ho_df

,srcLocInfo,dstLocInfo,nbHO15,srcTechno,srcLON,srcLAT,dstTechno,dstLON,dstLAT
index,,,,,,,,,
3240,8102f8100000994e,8102f81000009952,174,4G,4.818062,45.762236,4G,4.827545,45.752980
3241,8102f8100000994e,8102f81000009957,3879,4G,4.818062,45.762236,4G,4.826112,45.774444
3242,8102f8100000994e,8102f81000009958,1163,4G,4.818062,45.762236,4G,4.835282,45.761386
3243,8102f8100000994e,8102f8100000995b,17,4G,4.818062,45.762236,4G,4.858331,45.753607
3245,8102f8100000994e,8102f81000009963,553,4G,4.818062,45.762236,4G,4.830520,45.753373
...,...,...,...,...,...,...,...,...,...
20194,8102f8100000bad0,8102f81000009b28,64825,4G,4.820179,45.775520,4G,4.819723,45.771390
20195,8102f8100000bad0,8102f8100000a01d,5663,4G,4.820179,45.775520,4G,4.789162,45.763888
20196,8102f8100000bad0,8102f8100000a104,15,4G,4.820179,45.775520,4G,4.863723,45.755097


In [96]:
PATH_TOPO = '..\\..\\Data_Handover\\TOPO\\Lyon\\part-00000-2f038a97-7faf-46f3-aee2-b73b52bf2cba.snappy.parquet'
enb_df = pd.read_parquet(PATH_TOPO, engine='fastparquet')
enb_df = enb_df.loc[enb_df['LocInfo'].isin(ho_df['srcLocInfo'])].reset_index(drop=True)
enb_df

,LocInfo,TECHNO,LON,LAT
0,8102f8100000999d,4G,4.888138,45.738378
1,8102f81000009b28,4G,4.819723,45.771390
2,8102f8100000997e,4G,4.875561,45.753615
3,8102f81000009976,4G,4.846672,45.729169
4,8102f81000009959,4G,4.822966,45.733027
...,...,...,...,...
192,8102f81000009f4a,4G,4.874679,45.764310
193,8102f8100000997b,4G,4.880686,45.725675
194,8102f8100000ba70,4G,4.867783,45.701943
195,8102f8100000996d,4G,4.864943,45.729614


### Ground-truth data

In [5]:
PATH_GT = '..\\..\\Data_Handover\\GroundTruth\\GROUNDTRUTH_'

bike_df = pd.read_parquet(PATH_GT + 'BICYCLE.snappy.parquet', engine='fastparquet')
car_df = pd.read_parquet(PATH_GT + 'CAR_1.snappy.parquet', engine='fastparquet')
tcl_df = pd.read_parquet(PATH_GT + 'TCL_1.snappy.parquet', engine='fastparquet')
bike_df

,LAT,LON
0,45.734265,4.804213
1,45.734325,4.803973
2,45.734300,4.803960
3,45.734170,4.803900
4,45.733809,4.803814
...,...,...
177,45.759297,4.852736
178,45.759364,4.853488
179,45.759442,4.854239
180,45.759532,4.855359


## 2. Assign node and flow attributes

In [22]:
ENB_SRC = '8102f8100000ba53' 
ENB_DST = '8102f8100000a0bc'

### Nodes: geographic ranking

In [49]:
def assign_georank(enb_df, src_id=ENB_SRC, precision=1000):
    cells = np.array(enb_df[['LAT','LON']])
    centroid = np.array(enb_df[['LAT','LON']].loc[enb_df['LocInfo']==src_id]) 
    distances = np.linalg.norm(cells-centroid, axis=1) * precision   # normalized distances to centroid, between 0 and 100
    enb_df['Georank'] = [floor(d) for d in distances] 
    return enb_df

In [97]:
enb_df = assign_georank(enb_df)
enb_df

,LocInfo,TECHNO,LON,LAT,Georank
0,8102f8100000999d,4G,4.888138,45.738378,82
1,8102f81000009b28,4G,4.819723,45.771390,37
2,8102f8100000997e,4G,4.875561,45.753615,72
3,8102f81000009976,4G,4.846672,45.729169,42
4,8102f81000009959,4G,4.822966,45.733027,18
...,...,...,...,...,...
192,8102f81000009f4a,4G,4.874679,45.764310,74
193,8102f8100000997b,4G,4.880686,45.725675,76
194,8102f8100000ba70,4G,4.867783,45.701943,71
195,8102f8100000996d,4G,4.864943,45.729614,60


### Flows: handover weight

In [169]:
dist_ser = pd.Series()
for idx,row in ho_df.iterrows():
    dist_ser[idx] = max(0.1, dist((row.srcLAT,row.srcLON), (row.dstLAT,row.dstLON))*10000)
ho_df['Distance'] = dist_ser

C:\Users\arcti\AppData\Local\Temp\ipykernel_3468\832135131.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ho_df['Distance'] = dist_ser


In [173]:
ho_df['Weight'] = ho_df.Distance / ho_df.nbHO15
ho_df

C:\Users\arcti\AppData\Local\Temp\ipykernel_3468\1061336190.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ho_df['Weight'] = ho_df.Distance / ho_df.nbHO15


,srcLocInfo,dstLocInfo,nbHO15,srcTechno,srcLON,srcLAT,dstTechno,dstLON,dstLAT,Weight,Distance
index,,,,,,,,,,,
3240,8102f8100000994e,8102f81000009952,174,4G,4.818062,45.762236,4G,4.827545,45.752980,0.761577,132.514461
3241,8102f8100000994e,8102f81000009957,3879,4G,4.818062,45.762236,4G,4.826112,45.774444,0.037698,146.231927
3242,8102f8100000994e,8102f81000009958,1163,4G,4.818062,45.762236,4G,4.835282,45.761386,0.148246,172.409658
3243,8102f8100000994e,8102f8100000995b,17,4G,4.818062,45.762236,4G,4.858331,45.753607,24.225383,411.831519
3245,8102f8100000994e,8102f81000009963,553,4G,4.818062,45.762236,4G,4.830520,45.753373,0.276474,152.890331
...,...,...,...,...,...,...,...,...,...,...,...
20194,8102f8100000bad0,8102f81000009b28,64825,4G,4.820179,45.775520,4G,4.819723,45.771390,0.000641,41.550976
20195,8102f8100000bad0,8102f8100000a01d,5663,4G,4.820179,45.775520,4G,4.789162,45.763888,0.058496,331.263900
20196,8102f8100000bad0,8102f8100000a104,15,4G,4.820179,45.775520,4G,4.863723,45.755097,32.063679,480.955181


In [172]:
3879/146

26.568493150684933

In [ ]:
ho_max = ho_df['nbHO15'].max()
#ho_df['Weight'] = 1/ho_df['nbHO15']  # works okay
#ho_df['Weight'] = (ho_max - ho_df['nbHO15'])*0.001  # weird results
ho_df

C:\Users\arcti\AppData\Local\Temp\ipykernel_3468\1045963520.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ho_df['Weight'] = (ho_max - ho_df['nbHO15'])*0.001


,srcLocInfo,dstLocInfo,nbHO15,srcTechno,srcLON,srcLAT,dstTechno,dstLON,dstLAT,Weight
index,,,,,,,,,,
3240,8102f8100000994e,8102f81000009952,174,4G,4.818062,45.762236,4G,4.827545,45.752980,419.282
3241,8102f8100000994e,8102f81000009957,3879,4G,4.818062,45.762236,4G,4.826112,45.774444,415.577
3242,8102f8100000994e,8102f81000009958,1163,4G,4.818062,45.762236,4G,4.835282,45.761386,418.293
3243,8102f8100000994e,8102f8100000995b,17,4G,4.818062,45.762236,4G,4.858331,45.753607,419.439
3245,8102f8100000994e,8102f81000009963,553,4G,4.818062,45.762236,4G,4.830520,45.753373,418.903
...,...,...,...,...,...,...,...,...,...,...
20194,8102f8100000bad0,8102f81000009b28,64825,4G,4.820179,45.775520,4G,4.819723,45.771390,354.631
20195,8102f8100000bad0,8102f8100000a01d,5663,4G,4.820179,45.775520,4G,4.789162,45.763888,413.793
20196,8102f8100000bad0,8102f8100000a104,15,4G,4.820179,45.775520,4G,4.863723,45.755097,419.441


## 3. NetworkX graphs

In [174]:
G = nx.from_pandas_edgelist(ho_df, source='srcLocInfo', target='dstLocInfo', edge_attr='Weight', create_using=nx.DiGraph)

In [175]:
sp = nx.shortest_path(G, ENB_SRC, ENB_DST, weight='Weight', method='bellman-ford') # Bellman-Ford is better for directed graphs
#sp = nx.shortest_path(G, ENB_SRC, ENB_DST, weight='Weight', method='dijkstra') 

In [176]:
sp

['8102f8100000ba53',
 '8102f81000009f17',
 '8102f8100000ba4d',
 '8102f81000009952',
 '8102f8100000999e',
 '8102f810000099ab',
 '8102f81000009958',
 '8102f81000009995',
 '8102f81000009968',
 '8102f8100000998b',
 '8102f8100000bab9',
 '8102f810000099a6',
 '8102f81000009953',
 '8102f8100000a0bc']

## 4. Folium visualization

In [165]:
def create_map():
    map = folium.Map([45.73303, 4.82297], tiles="OpenStreetMap", zoom_start=13)
    print('Creating base station map...')
    return map

def add_stations(map, df, name='4G stations'):
    print('Adding '+name+' layer')
    fg = folium.FeatureGroup(name=name) # Name as it will appear in Layer control
    enb_ids = df.srcLocInfo.unique()
    for id in enb_ids:
        enb_data = df.loc[df['srcLocInfo']==id]
        fg.add_child(folium.Marker(
            location=[enb_data['srcLAT'].iloc[-1], enb_data['srcLON'].iloc[-1]],
            popup=enb_data['srcLocInfo'].iloc[-1],
        ))
    map.add_child(fg)
    
def close_map(map, filename):
    folium.LayerControl().add_to(map)
    map.save('maps\\'+filename)
    print('Closing', filename, 'map.')
    
def add_voronoi(map, points):
    print('Adding Voronoi layer')
    vor = Voronoi(list(zip(points['LAT'].tolist(), points['LON'].tolist())))  # Careful with the inversion!!  
    fg = folium.FeatureGroup(name='Voronoi cells', show=False)
    for enb, reg_idx in enumerate(vor.point_region):
        region = vor.regions[reg_idx]
        if -1 not in region and region != []:
            region_coords = []
            for vertex in region:
                region_coords.append(vor.vertices[vertex])
            fg.add_child(folium.Polygon(
                region_coords,
                color="black",
                weight=1,
                fill_color="darkgray",
                fill_opacity=0.2,
                fill=True,
            ))
    map.add_child(fg)
    
    
def add_flows(map, data, name='4G Handovers', min_weight=900):
    print('Adding Flow layer', name)
    fg = folium.FeatureGroup(name=name)
    flow_weight = {100000:2, 200000:4, 300000:6, 400000:8, 500000:10}
    flow_color = {2:'blue', 4:'green', 6:'yellow', 8:'orange', 10:'red'}
    for idx, row in data.iterrows():
        if row['nbHO15'] >= min_weight:
            fw = [v for k,v in flow_weight.items() if row['nbHO15'] <= k]
            fg.add_child(folium.PolyLine(
                [(row['srcLAT'], row['srcLON']), (row['dstLAT'], row['dstLON'])],
                color = flow_color[fw[0]],
                weight = fw[0],
                opacity = 0.8,
                tooltip = 'Weight: '+str(row['nbHO15'])
            ))
    map.add_child(fg)
    
    
def add_path(map, data, path_list, name='Path', color='#AA0000'):
    print('Adding Path layer', name)
    fg = folium.FeatureGroup(name=name)
    for i in range(1, len(path_list)):
        flow = data.loc[(data['srcLocInfo']==path_list[i-1]) & (data['dstLocInfo']==path_list[i])]
        fg.add_child(folium.PolyLine(
            locations=[[flow['srcLAT'].iloc[0], flow['srcLON'].iloc[0]], [flow['dstLAT'].iloc[0], flow['dstLON'].iloc[0]]],
            color = color,
            opacity = 0.8,
            weight = 5,
            tooltip = 'nbHO: '+str(flow['nbHO15'].iloc[0])+'\nWeight: '+str(flow['Weight'].iloc[0])
        ))
    map.add_child(fg)
    
    
def add_voronoi_path(map, enb_data, cells_data, name='Cells path', color='green'):
    vor = Voronoi(list(zip(enb_data['LAT'].tolist(), enb_data['LON'].tolist())))  # Careful with the inversion!!  
    fg = folium.FeatureGroup(name=name, show=True)
    for enb_idx, reg_idx in enumerate(vor.point_region):
        region = vor.regions[reg_idx]
        cell_locinfo = enb_data.LocInfo.iloc[enb_idx]
        if -1 not in region and region != []:
            region_coords = []
            if type(cells_data) == list:
                if cell_locinfo in cells_data:
                    for vertex in region:
                        region_coords.append(vor.vertices[vertex])
                    fg.add_child(folium.Polygon(
                        region_coords,
                        color = color,
                        weight=1,
                        fill_color = color,
                        fill_opacity=0.3,
                        fill=True,
                    ))
            else:
                if cell_locinfo in cells_data['LocInfo'].values:
                    for vertex in region:
                        region_coords.append(vor.vertices[vertex])
                    fg.add_child(folium.Polygon(
                        region_coords,
                        color = color,
                        weight=1,
                        fill_color = color,
                        fill_opacity=0.3,
                        fill=True,
                    ))
    map.add_child(fg)

In [ ]:
lyon = create_map()
add_stations(lyon, ho_df)
add_voronoi(lyon, enb_df)

#add_flows(lyon, ho_df.loc[ho_df.srcLocInfo.isin(sp)])
add_voronoi_path(lyon, enb_df, cells_data=sp, name='Cells path', color='green')
add_path(lyon, ho_df, path_list=sp)
close_map(lyon, 'lyon_shortest_path.html')
lyon

Creating base station map...
Adding 4G stations layer
Adding Voronoi layer
Adding Path layer Path
Closing lyon_shortest_path.html map.
